# Import libraries and data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
train_essays = pd.read_csv('train_essays.csv')
train_essays.columns

Index(['id', 'prompt_id', 'text', 'generated'], dtype='object')

Here ID and prompt id are not required, hence drop them from the data

In [2]:
train_essays = train_essays.drop(['id','prompt_id'],axis=1)
train_essays['generated'].value_counts()

0    1375
1       3
Name: generated, dtype: int64

In [3]:
train_essays.columns

Index(['text', 'generated'], dtype='object')

# Balancing the data/ oversampling

So Only 2 essays are AI Generated and the remaining are Human Essays, so we need to make the number of AI Generated Essays and Human Essays equal

In [4]:
# new_data = pd.read_csv('train_v2_drcat_02.csv')
# selected_data = new_data[new_data['label'] == 1].sample(n=600, random_state=42)[['text', 'label']]
# selected_data.to_csv('ai_generated_data.csv')
ai_generated_data = pd.read_csv('ai_generated_data.csv')
ai_generated_data = ai_generated_data.drop('Unnamed: 0',axis=1)
ai_generated_data.rename(columns={'label': 'generated'}, inplace=True)


In [5]:
ai_generated_data.columns

Index(['text', 'generated'], dtype='object')

# Merging the dataset into one dataset

In [6]:

df = pd.concat([train_essays, ai_generated_data], ignore_index=True)


In [7]:
df.shape, df.columns

((1978, 2), Index(['text', 'generated'], dtype='object'))

In [8]:
df['generated'].value_counts()

0    1375
1     603
Name: generated, dtype: int64

In [9]:
train_data, dev_data = train_test_split(df, test_size=0.2, random_state=42)
train_data.shape, dev_data.shape


((1582, 2), (396, 2))

# Buliding a vocabulary list

In [10]:
from collections import Counter

# Create a list of all words in the dataset
all_words = ' '.join(df['text']).lower().split()

# Count word occurrences
word_counts = Counter(all_words)

# Build vocabulary
vocab = [word for word, count in word_counts.items() if count >= 5]
vocab_reverse_index = {word: idx for idx, word in enumerate(vocab)}

# Calculating the word probability

In [11]:
# Calculate P[word]
num_documents = len(df)
word_probabilities = {word: count / num_documents for word, count in word_counts.items()}

human_data = df[df['generated'] == 0]
llm_data = df[df['generated'] == 1]

# Calculate P[word | LLM]
llm_word_probabilities = {word: df['text'].apply(lambda essay: word in essay.lower()).mean() for word in vocab}


In [12]:
llm_word_probabilities

{'cars.': 0.21941354903943378,
 'cars': 0.38877654196157735,
 'have': 0.9110212335692619,
 'been': 0.41051567239635994,
 'around': 0.2826086956521739,
 'since': 0.2517694641051567,
 'they': 0.8038422649140546,
 'became': 0.023255813953488372,
 'famous': 0.016177957532861477,
 'in': 0.9989888776541962,
 'the': 1.0,
 'when': 0.621840242669363,
 'henry': 0.0025278058645096056,
 'ford': 0.1552072800808898,
 'created': 0.0955510616784631,
 'and': 0.9989888776541962,
 'built': 0.04095045500505561,
 'first': 0.2992922143579373,
 'played': 0.010616784630940344,
 'a': 1.0,
 'major': 0.3023255813953488,
 'role': 0.05055611729019211,
 'our': 0.871587462082912,
 'every': 0.4929221435793731,
 'day': 0.4929221435793731,
 'lives': 0.16279069767441862,
 'then.': 0.005561172901921132,
 'but': 0.782608695652174,
 'now,': 0.08139534883720931,
 'people': 0.8033367037411526,
 'are': 0.9565217391304348,
 'starting': 0.05358948432760364,
 'to': 0.9984833164812943,
 'question': 0.06268958543983821,
 'if': 0.9

# Calculating the accuracy of the dev dataset

In [14]:
# Function to classify essays based on probabilities
def classify_essays(essays, word_probabilities, llm_word_probabilities, vocab):
    predictions = []

    for essay in essays:
        # Tokenize essay into words
        words = essay.lower().split()

        # Initialize probabilities for each class
        prob_human = 1.0
        prob_llm = 1.0

        for word in words:
            # Check if the word is in the vocabulary
            if word in vocab:
                # Calculate P(word | Human) using Laplace smoothing
                prob_word_human = (word_probabilities.get(word, 0) + 1) / (len(vocab) + len(words))

                # Calculate P(word | LLM) using Laplace smoothing
                prob_word_llm = (llm_word_probabilities.get(word, 0) + 1) / (len(vocab) + len(words))

                # Update class probabilities
                prob_human *= prob_word_human
                prob_llm *= prob_word_llm

        # Classify based on the probabilities
        prediction = 1 if prob_llm > prob_human else 0
        predictions.append(prediction)

    return predictions

# Apply the classifier on the dev dataset
dev_predictions = classify_essays(dev_data['text'], word_probabilities, llm_word_probabilities, vocab)

# Calculate accuracy
accuracy = (dev_predictions == dev_data['generated']).mean()
print(f"Accuracy on dev dataset: {accuracy}")


Accuracy on dev dataset: 0.6843434343434344


# Doing Experiments

In [15]:
import numpy as np

# Laplace Smoothing function
def laplace_smoothing(count, total_count, vocab_size, alpha=1):
    return (count + alpha) / (total_count + alpha * vocab_size)

# Implement Naive Bayes Classifier with Laplace Smoothing
def classify_essays_with_smoothing_probabilities(essays, word_probabilities, llm_word_probabilities, vocab, alpha=1):
    probabilities = []

    for essay in essays:
        # Initialize probabilities
        human_prob = 0.0
        llm_prob = 0.0

        for word in essay.lower().split():
            if word in vocab:
                human_prob += np.log(laplace_smoothing(word_probabilities[word], len(train_data), len(vocab), alpha))
                llm_prob += np.log(laplace_smoothing(llm_word_probabilities[word], len(llm_data), len(vocab), alpha))

        # Apply sigmoid function to get probabilities in decimal
        human_prob = 1 / (1 + np.exp(-human_prob))
        llm_prob = 1 / (1 + np.exp(-llm_prob))

        probabilities.append({'human_prob': human_prob, 'llm_prob': llm_prob})

    return probabilities

# Apply the classifier with Laplace smoothing on the dev dataset
dev_probabilities_with_smoothing = classify_essays_with_smoothing_probabilities(dev_data['text'], word_probabilities, llm_word_probabilities, vocab, alpha=1)

# Convert probabilities to binary predictions
dev_predictions_with_smoothing = [1 if prob['llm_prob'] > prob['human_prob'] else 0 for prob in dev_probabilities_with_smoothing]

# Calculate accuracy
accuracy_with_smoothing = (dev_predictions_with_smoothing == dev_data['generated']).mean()
print(f"Accuracy on dev dataset with Laplace smoothing: {accuracy_with_smoothing}")

# Top 10 words predicting each class
top_10_human_words = sorted(vocab, key=lambda word: -np.log(laplace_smoothing(word_probabilities[word], len(train_data), len(vocab))))
top_10_llm_words = sorted(vocab, key=lambda word: -np.log(laplace_smoothing(llm_word_probabilities[word], len(llm_data), len(vocab))))

print("\nTop 10 words predicting Human essays:")
print(top_10_human_words[:10])

print("\nTop 10 words predicting LLM essays:")
print(top_10_llm_words[:10])


C:\Users\kashy\AppData\Local\Temp\ipykernel_24072\3712460965.py:22: RuntimeWarning: overflow encountered in exp
  human_prob = 1 / (1 + np.exp(-human_prob))
C:\Users\kashy\AppData\Local\Temp\ipykernel_24072\3712460965.py:23: RuntimeWarning: overflow encountered in exp
  llm_prob = 1 / (1 + np.exp(-llm_prob))


Accuracy on dev dataset with Laplace smoothing: 0.6843434343434344

Top 10 words predicting Human essays:
['the', 'to', 'of', 'and', 'a', 'in', 'is', 'that', 'for', 'it']

Top 10 words predicting LLM essays:
['the', 'a', 'i', 'he', 't', 'u', 'c', 'on', 'or', 'b']


# Kaggle Submission

In [17]:
# Apply the classifier with Laplace smoothing on the test dataset

test_data = pd.read_csv('test_essays.csv')
test_predictions = classify_essays_with_smoothing_probabilities(test_data['text'], word_probabilities, llm_word_probabilities, vocab, alpha=1)

prob_ = []
for dict1 in test_predictions:
    prob_.append(dict1['llm_prob'])
# Prepare the Kaggle submission
kaggle_submission = pd.DataFrame({'id': test_data['id'], 'generated': prob_})

# Save the submission to a CSV file
kaggle_submission.to_csv('kaggle_submission.csv', index=False)

# Print the submission
print(kaggle_submission)




         id  generated
0  0000aaaa        0.5
1  1111bbbb        0.5
2  2222cccc        0.5
